In [1]:
import numpy as np 
import pandas as pd 

In [2]:
dfe = pd.read_csv('r490.csv')
dfe = dfe[dfe['r1_charge_heater'] > 0]

In [4]:
target_cols = ['process_ron', 'process_yield', 'r1_charge_heater']
target_modes = [np.argmax, np.argmax, np.argmin]

monitored_cols = target_cols + ['r1_temp', 'r1_pressure', 'r2_temp', 'r2_pressure']

ixs = []
for target_col, target_mode in zip(target_cols, target_modes):
    ix_best = target_mode(dfe[target_col])
    ixs.append(ix_best)

global_sol = dfe.iloc[ixs][monitored_cols].copy()
global_sol.index = target_cols
global_sol.index.rename('optimization target', inplace=True)
global_sol['optimization mode'] = [tm.__name__.replace('arg','') for tm in target_modes]
global_sol = global_sol[['optimization mode'] + list(global_sol.columns)[:-1]]
global_sol['r1_charge_heater'] /= 1000000
global_sol.round(3)

,optimization mode,process_ron,process_yield,r1_charge_heater,r1_temp,r1_pressure,r2_temp,r2_pressure
optimization target,,,,,,,,
process_ron,max,83.775,97.763,3.380,138.0,50.0,142.0,70.0
process_yield,max,83.224,98.195,0.051,132.0,90.0,128.0,90.0
r1_charge_heater,min,83.246,98.183,0.005,132.0,60.0,128.0,60.0


In [5]:
max_ron = np.max(global_sol['process_ron'])
max_yield = np.max(global_sol['process_yield'])
min_ch = np.min(global_sol['r1_charge_heater'])

global_sol['ron_loss_perc'] = (max_ron - global_sol['process_ron']) * 100 / max_ron
global_sol['yield_loss_perc'] = (max_yield - global_sol['process_yield']) * 100 / max_yield
global_sol['ch_increase_ratio'] = global_sol['r1_charge_heater']/ min_ch
global_sol.round(2)

,optimization mode,process_ron,process_yield,r1_charge_heater,r1_temp,r1_pressure,r2_temp,r2_pressure,ron_loss_perc,yield_loss_perc,ch_increase_ratio
optimization target,,,,,,,,,,,
process_ron,max,83.78,97.76,3.38,138.0,50.0,142.0,70.0,0.00,0.44,679.62
process_yield,max,83.22,98.19,0.05,132.0,90.0,128.0,90.0,0.66,0.00,10.18
r1_charge_heater,min,83.25,98.18,0.00,132.0,60.0,128.0,60.0,0.63,0.01,1.00


In [6]:
global_sol.to_csv("tmp/table2.csv")